In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers import GRU
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LayerNormalization

dt='float32'
tf.keras.backend.set_floatx(dt)

EPOCHS=100
SPLITS=1
K=4
VOCABULARY_SIZE=4**K+1   # e.g. K=3 => 64 DNA K-mers + 'NNN'
EMBED_DIMEN=16

In [2]:
revived = tf.keras.models.load_model('embed_27.model')

In [3]:
revived.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1200, 16)          4112      
_________________________________________________________________
gru_2 (GRU)                  (None, 1200, 16)          1632      
_________________________________________________________________
gru_3 (GRU)                  (None, 16)                1632      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 7,393
Trainable params: 7,393
Non-trainable params: 0
_________________________________________________________________


In [32]:
#tensorflow.python.keras.engine.sequential.Sequential
type(revived)
#help(revived)
revived.count_params()
revived.get_input_at(0)
#help(embedder)
w=revived.get_layer(index=0).weights
#help(w[0])
w[0]

<tf.Variable 'embedding_1/embeddings:0' shape=(257, 16) dtype=float32, numpy=
array([[ 0.24055451, -0.8092862 , -0.13454944, ...,  0.08447518,
        -0.05890008, -0.08199664],
       [-0.43518454, -0.03887598, -1.6261969 , ...,  0.5823484 ,
        -0.14375973, -0.54512376],
       [-0.02438883,  0.28650796, -0.8032899 , ..., -2.5489483 ,
        -0.3846975 , -0.8488031 ],
       ...,
       [-0.03546799,  0.16184689,  1.7656252 , ...,  1.0695041 ,
        -2.0512128 , -0.8709499 ],
       [-0.4348466 ,  5.981366  ,  0.31181216, ...,  1.0816721 ,
         0.43400377, -2.16507   ],
       [-1.818359  , -1.0557351 , -0.64156395, ...,  0.7280673 ,
        -1.3553829 , -1.2423927 ]], dtype=float32)>

In [33]:
#np.savetxt("weights.csv", w, delimiter=",")
w[0][0]

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([ 0.24055451, -0.8092862 , -0.13454944, -0.58963734, -0.8864087 ,
        0.6779338 , -0.49168026,  0.8520491 , -0.610975  , -0.38268387,
        0.32842866, -0.71309805,  0.7131573 ,  0.08447518, -0.05890008,
       -0.08199664], dtype=float32)>

In [34]:
w[0][0][0]

<tf.Tensor: shape=(), dtype=float32, numpy=0.24055451>

In [48]:
ary2d=w[0].numpy()
ary2d
np.savetxt("embedded.csv", ary2d, delimiter=",")

In [45]:
for row in range(257):

    rowary = w[0][row].numpy()
    print(rowary)
    
#    colvals=[]
#    for col in range(16):
#        colval=str(w[0][row][col])
#        print (w[0][row][col])

#        colvals.append(colval)
#    rowstr=",".join(colvals)
#    print(rowstr)


[ 0.24055451 -0.8092862  -0.13454944 -0.58963734 -0.8864087   0.6779338
 -0.49168026  0.8520491  -0.610975   -0.38268387  0.32842866 -0.71309805
  0.7131573   0.08447518 -0.05890008 -0.08199664]
[-0.43518454 -0.03887598 -1.6261969   0.4989389   0.8410085   1.2635448
  1.0081367   0.8439172   1.0882741  -0.83177876  1.9759601   1.401775
  1.5558887   0.5823484  -0.14375973 -0.54512376]
[-0.02438883  0.28650796 -0.8032899   0.2696225  -2.2867367   0.9896725
  0.6763644  -0.22041236 -0.25986636 -0.8062884  -0.29467997 -0.1902887
  0.5184637  -2.5489483  -0.3846975  -0.8488031 ]
[ 0.5520751   1.8597528   0.6764902  -0.44897196 -2.0028777  -1.2711924
  0.36815232 -0.03958852 -0.08606648 -0.00810473 -2.0123746   0.41409212
 -0.49125263  0.16988695  0.43819785 -1.3801336 ]
[ 0.9031857   0.6158776  -0.39416277 -1.1561762  -2.8083727   1.3260751
  1.4103966   0.05527422 -0.3391862  -0.28027767  0.38042897 -1.4053589
  3.3666391  -3.1844451  -3.7467659  -4.040463  ]
[-0.3711165   0.1882277   0.1

 -0.8598825   0.1682274   0.03038497  0.7582145 ]
[-1.1646265   0.45597684 -1.2431246   0.05219448  3.1284878   0.76747257
  0.8400352  -0.6861874   0.05163874  1.1528364   1.239783   -2.9845116
 -0.5613129   0.26866233  1.9308901  -3.0904047 ]
[ 0.80346656  1.0644609  -2.8365705  -0.425363    3.2132485   2.444712
  0.44538796 -2.0997612   3.447371   -1.9938916   4.1627145   0.6562094
  0.6328154   2.3812308   0.93315655  0.94031096]
[ 0.63029385 -0.0518462  -2.6249669   1.018017    2.0899906   2.557704
  0.86832184 -2.5024993   2.251609   -0.33866572  3.8237128   1.2571057
 -0.40506902 -0.33364096 -1.1726335   2.8284307 ]
[-0.07303763  0.44379863 -1.4607917   0.866687    3.0818884  -0.32127964
 -0.622773   -2.9019022  -0.1828499   0.19654153  1.2487303   0.185868
 -0.97807384 -1.3800305   3.860197    1.0692987 ]
[-4.1136928  -1.374025   -4.134237    0.8836167   3.9581163   1.0657599
 -1.66042    -0.28787044 -0.2939754   0.58916     5.524174   -2.3146422
  2.656096    0.11673568  1.050